In [1]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [1]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib
importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Petra')
domain = await ui.choose_domain(domain='grammar')
task_type = await ui.choose_task_type(task_type='multiple_choice')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a multiple choice with given answer possibilities, there can be multiple correct answers. Generate only one task with four answer possibilities. Generate it for the area of grammar teaching, targeting on To bend the rules at my own will.
Adjust the task dificulty to language level B1 (Can understand the main points of clear standard input on familiar matters regularly encountered in work, school, leisure, etc. Can deal with most situations likely to arise whilst travelling in an area where the language is spoken.  Can produce simple connected text on topics which are familiar or of personal interest. Can describe experiences and events, dreams, hopes & ambitions and briefly give reasons and explanations for opinions and plans).

The following are 2 examples of how the task should be generated with output format and style:

 Sele

In [5]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Select the correct answers:
I want to ______ the rules at my own will.
a) bend
b) breaking
c) bends
d) broke


In [7]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="the correct answer is a")

prompt = await evaluator.generate_prompt(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print(prompt)


NameError: name 'formated_user_answer' is not defined

In [13]:
######################
# This is what it will actually looks like
#NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(prompt)
#print('NL_feedback =', NL_feedback)
#print('correctness =', correctness)
#print('training_goals =', training_goals)

#########################
# This is how it will be implemented inside the evaluate_learner_answer function

print(connector_openai.text_completion(prompt))

1) False
2) Vocabulary: rules, individuality, conformity
   Grammar: verb forms (rebel, live)
   Text skills: understanding context, coherence
3) Your answer for the first gap is incorrect. The appropriate word to fill in the gap is "rebel". This word means to resist or defy the rules or authority. You should practice your vocabulary related to rules and individuality. For the second gap, the correct word is "break". This word means to go against or violate the rules. Lastly, for the third gap, the correct word is "live". It seems you misinterpreted the meaning of this sentence. The sentence means that you will live life on your own terms and in your own way. You should practice understanding the context of the text and improving your coherence when filling in the gaps. Keep practicing and don't be afraid to ask for help!


In [8]:
new_profile = await evaluator.update_learner_profile(training_goals, profile)
print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

NameError: name 'training_goals' is not defined

In [9]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
# await db.save_profile(new_profile)